<a href="https://colab.research.google.com/github/genniebean48/MotorcycleDatabase/blob/master/Zero_Motorcycle_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>**Zero Motorcycle Data Analysis </center>**
[<center> https://www.zeromotorcycles.com/](https://www.zeromotorcycles.com/</center>)


![alt text](http://www.logo-designer.co/wp-content/uploads/2018/06/2018-zero-motorcycles-new-logo-design-2.png)




In [0]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

import io

df2 = pd.read_csv(io.BytesIO(uploaded['Charging.csv']))


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## **Total Number of Events**
This SQL query counts the number of events in each table. The first row is the number of Riding events, the second is the number of Charging events, then Debug events, then Disarmed events, then key events, then Info events, then Power events and finally Sevcon events. 

In [0]:
import sqlite3
bikesdb = sqlite3.connect('/content/Bikes.DB')
table = bikesdb.execute('SELECT Count(event), "Riding" FROM Events WHERE event = "Riding"')# Union SELECT Count(event) FROM Events WHERE event = "Charging" Union SELECT Count(event) FROM Events WHERE event = "DEBUG:" Union SELECT Count(event) FROM Events WHERE event = "Disarmed" Union SELECT Count(event) FROM Events WHERE event = "Key" Union SELECT Count(event) FROM Events WHERE event = "INFO:" Union SELECT Count(event) FROM Events WHERE event = "Power" Union SELECT Count(event) FROM Events WHERE event = "Sevcon"')
data = table.fetchall()

for row in data:
    row = str(row).strip('(),')
    print(row)     


41270, 'Riding'


## Average Ride Distance
This query returns the average distance a motorcycle covers for the duration of a ride. This distance is measured in kilometers. The query takes into account and separates the different ride events in a single file. 



In [0]:
import sqlite3
bikesdb = sqlite3.connect('/content/Bikes.DB')
table = bikesdb.execute('SELECT avg(Odometer) FROM(SELECT Vin, RideNum, count(Sequence), (max(Odo) - min(Odo)) as "Odometer" FROM Riding GROUP BY Vin, RideNum)')
data = table.fetchall()

for row in data:
    row = str(row).strip('(),')
    row = float(row)
    row = round(row, 2)
    row = str(row) + 'km'
    print(row)

32.26km


## **Average State of Charge**
This section of code calculates the average state of charge a zero motorcycle maintains during their Riding and Disarmed events. The first row of the table is the average state of charge for all of the Riding events, and the second row is the average state of charge for all of the Disarmed events. 

In [0]:
import sqlite3
bikesdb = sqlite3.connect('/content/Bikes.DB')
table = bikesdb.execute('SELECT avg(PackSOC) FROM Riding UNION SELECT avg(PackSOC) FROM Disarmed')
data = table.fetchall()

for row in data:
    row = str(row).strip('(),' + 'column')
    row = float(row)
    row = round(row, 2)
    row = str(row) + ('%')
    print(row)

72.23%
72.94%


## Average SOC After a Charging Session
This SQL query calculates the average SOC user charges their bikes back up. This code separates the charging series from each of the files to take into account the multiple charging session per file/motorcycle.


In [0]:
import sqlite3
bikesdb = sqlite3.connect('/content/Bikes.DB')
table = bikesdb.execute('SELECT avg(Charge) FROM(SELECT Vin, ChargeNum, count(Sequence), max(PackSOC) as "Charge" FROM Charging GROUP BY Vin, ChargeNum)')
data = table.fetchall()

for row in data:
    row = str(row).strip('(),')
    row = float(row)
    row = round(row, 2)
    row = str(row) + ('%')
    print(row)

82.71%


## Average Ride Distance For Each Month
This SQL query is calculates the average ride distance in a given month. This is aimed to research whether or not riders will go farther in the summer time vs. the winter, vs. the spring or the fall. 